# Résolution problème Osborne 2

In [ ]:
using BenchmarkTools, ForwardDiff, Distributions

In [ ]:
include("../src/enlsip_functions_castor2.jl")

In [ ]:
n = 11
m = 65 
nb_eq = 0
nb_constraints = 22

In [ ]:
# DataPoints

dataset = [1 0.0 1.366 ;
2 0.1 1.191 ;
3 0.2 1.112 ;
4 0.3 1.013 ;
5 0.4 0.991 ;
6 0.5 0.885 ;
7 0.6 0.831 ;
8 0.7 0.847 ;
9 0.8 0.786 ;
10 0.9 0.725 ;
11 1.0 0.746 ;
12 1.1 0.679 ;
13 1.2 0.608 ;
14 1.3 0.655 ;
15 1.4 0.616 ;
16 1.5 0.606 ;
17 1.6 0.602 ;
18 1.7 0.626 ;
19 1.8 0.651 ;
20 1.9 0.724 ;
21 2.0 0.649 ;
22 2.1 0.649 ;
23 2.2 0.694 ;
24 2.3 0.644 ;
25 2.4 0.624 ;
26 2.5 0.661 ;
27 2.6 0.612 ;
28 2.7 0.558 ;
29 2.8 0.533 ;
30 2.9 0.495 ;
31 3.0 0.500 ;
32 3.1 0.423 ;
33 3.2 0.395 ;
34 3.3 0.375;
35 3.4 0.538 ;
36 3.5 0.522 ;
37 3.6 0.506 ;
38 3.7 0.490 ;
39 3.8 0.478 ;
40 3.9 0.467 ;
41 4.0 0.457 ;
42 4.1 0.457 ;
43 4.2 0.457 ;
44 4.3 0.457 ;
45 4.4 0.457 ;
46 4.5 0.457 ;
47 4.6 0.457 ;
48 4.7 0.457 ;
49 4.8 0.457 ;
50 4.9 0.457 ;
51 5.0 0.457;
52 5.1 0.431 ;
53 5.2 0.431 ;
54 5.3 0.424 ;
55 5.4 0.420 ;
56 5.5 0.414 ;
57 5.6 0.411 ;
58 5.7 0.406 ;
59 5.8 0.406 ;
60 5.9 0.406 ;
61 6.0 0.406 ;
62 6.1 0.406 ;
63 6.2 0.406 ;
64 6.3 0.406 ;
65 6.4 0.406]

t = dataset[1:m,2]
y = dataset[1:m,3]

In [ ]:
function r_k(x::Vector, t::Float64, y::Float64)
    rx = x[1]*exp(-x[5]*t) + x[2]*exp(-x[6]*(t-x[9])^2) + x[3]*exp(-x[7]*(t-x[10])^2) + x[4]*exp(-x[8]*(t-x[11])^2)
    return y - rx
end

function r(x::Vector)
    return [r_k(x,t[k],y[k]) for k=1:m]
end

resOsborne2 = ResidualsEval(0)

function (resOsborne2::ResidualsEval)(x::Vector, rx::Vector, J::Matrix)

    # Evaluate the residuals
    if abs(resOsborne2.ctrl) == 1
        rx[:] = r(x)

    # The jacobian is computed analytically
    elseif resOsborne2.ctrl == 2
        J[:] = ForwardDiff.jacobian(r,x)
    end
    return
end

In [ ]:
function c(x::Vector)
    res = [x[1] - 1.31; 1.4 - x[1];
            x[2] - 0.4314 ; 0.8 - x[2];
            x[3] - 0.6336; 1.0 - x[3];
            x[4] - 0.5; 1.0 - x[4];
            x[5] - 0.5; 1.0 - x[5];
            x[6] - 0.6; 3.0 - x[6];
            x[7] - 1.0; 5.0 - x[7];
            x[8] - 4.0; 7.0 - x[8];
            x[9] - 2.0; 2.5 - x[9];
            x[10] - 4.5689; 5.0 - x[10];
            x[11] - 5.0; 6.0 - x[11]]
    return res
end
    
consOsborne2 = ConstraintsEval(0)

function (consOsborne2::ConstraintsEval)(x::Vector, cx::Vector, A::Matrix)

    # Evaluate the constraints
    if abs(consOsborne2.ctrl) == 1
        cx[:] = c(x)
    # The jacobian is computed numerically if ctrl is set to 0 on return
    elseif consOsborne2.ctrl == 2
        A[:] = ForwardDiff.jacobian(c,x)
    end
    return
end

In [ ]:
function generate_starting_point()
    x0 = [rand(Uniform(1.31,1.4));
        rand(Uniform(0.4314 , 0.8));
        rand(Uniform(0.6336, 1.0));
        rand(Uniform(0.5, 1.0));
        rand(Uniform(0.5, 1.0));
        rand(Uniform(0.6, 3.0));
        rand(Uniform(1.0, 5.0));
        rand(Uniform(4.0, 7.0));
        rand(Uniform(2.0, 2.5));
        rand(Uniform(4.5689, 5.0));
        rand(Uniform(5.0, 6.0))]
    return x0
end

In [ ]:
x_saved =  [1.3344098963722457
 0.5572842161127423
 0.6757364753061974
 0.8291980513226953
 0.9233565833014519
 0.9588470511477797
 1.9610314699563896
 4.055321823656234
 2.048625993866472
 4.60296578920499
 5.95212572157736]

In [ ]:
# ε_abs=1e-10, ε_rel=1e-5, ε_x=1e-3, ε_c=1e-4
# x0 = generate_starting_point()
x0= x_saved
e = eps(Float64)

enlsipOsborne2 = enlsip(x0,resOsborne2,consOsborne2,n,m,nb_eq,nb_constraints,verbose=true)

## Visualisation

In [ ]:
using Plots

In [ ]:
str_to_array = (str::String, T::DataType=Float64) -> parse.(T, split(chop(str; head=1, tail=1), ','))

df = DataFrame(CSV.File("iterates.csv", delim=";"))

In [ ]:
f = (x::Vector,t::Number) -> x[1]*exp(-x[5]*t) + x[2]*exp(-x[6]*(t-x[9])^2) + x[3]*exp(-x[7]*(t-x[10])^2) + x[4]*exp(-x[8]*(t-x[11])^2)

entry = range(0,7.0,1000)
anim = @animate for i ∈ df[!,:iter]
    scatter(t,y,markershape=:cross, xlabel="t", ylabel="y", legend=:none,ylims=(0,1.5))
    x = str_to_array(df[i,:x])
    plot!(entry, (t -> f(x,t)).(entry),title="Itération $i")
end
gif(anim, fps = 2)

In [ ]:
# Calcul du point le plus éloigné de la droite entre le point de départ et la solution


x_0 = str_to_array(first(df[!,:x]))
x_N = str_to_array(last(df[!,:x]))
xNmx0 = x_N - x_0
nrm_xNmx0 = norm(xNmx0)

dist = 0
x_loin = x_0
x_loin_proj = x_0

# println("||xN-x0|| = $nrm_xNmx0")

for str_x ∈ df[!,:x]
    x = str_to_array(str_x)
#     println("\n\ncoeff :",(dot(x-x_0,xNmx0) / nrm_xNmx0)," pour x = $x")
    x_proj = (dot(x-x_0,xNmx0) / dot(xNmx0,xNmx0)) * xNmx0 + x_0
#     println(x_proj)
#     println(dot(x-x_proj,xNmx0))
    dist_proj = norm(x-x_proj)
    if dist_proj > dist
        dist = dist_proj
        x_loin = x
        x_loin_proj = x_proj
    end
end

In [ ]:
using Plots; plotlyjs()

In [ ]:


# Square sum of residuals

function ssr(x::Vector)
    rx = r(x)
#     return min(10,dot(rx,rx))
    return dot(rx,rx)
end

function ssc(x::Vector)
    cx = c(x) 
    return dot(cx, (t->min(0,t)).(cx))
end

# Rays of the cone {λd + μd̄ | (λ,μ) ∈ [0,ρ]²}

d = x_N - x_0 # x_loin - (3x_0-x_N)/2
d̄ = x_loin - x_loin_proj
ρ = 2*nrm_xNmx0 #  norm(xNmx0*3/2)

λ_inf = -1. # -1.
λ_sup = 3. # 3.
μ_inf = -1. # -1.
μ_sup = 3. # 3.

scal_d = range(λ_inf,λ_sup,1000)
scal_d̄ = range(μ_inf,μ_sup,1000)

C =  [x_0 + λ*d + μ*d̄ for μ ∈ scal_d̄, λ ∈ scal_d]

z = (ssr).(C)
cz = (ssc).(C)
# @show extrema(z)


# Projection des itérés
iterates = str_to_array.(df[!,:x])
λ_iterates = [dot(x-x_0,xNmx0) * (1/dot(xNmx0,xNmx0)) for x ∈ iterates]
μ_iterates = [dot(x-x_0,d̄) * (1/dot(d̄,d̄)) for x ∈ iterates]
residuals_iterates = ssr.(iterates) 
zmax = 2*ceil(maximum(residuals_iterates))
N = length(iterates)

# Plot de la surface

# scatter3d!([1],[0],[enlsipOsborne2.obj_value],markershape=:diamond)

scatter3d(λ_iterates, μ_iterates, residuals_iterates, mz=range(1,N),mc=:blues,colorbar=:none)
plot!(λ_iterates, μ_iterates, residuals_iterates, mz=range(1,N),lw=3,lc=:green,colorbar=:none)
surface!(scal_d,scal_d̄, z, xlabel="λ", ylabel="μ", zlabel="Résidus", fz = cz, fc = :acton,
    xlims=(λ_inf,λ_sup), ylims=(μ_inf,μ_sup), zlims=(0,zmax),colorbar=:best)

# surface!(scal_d,scal_d̄, cz)



# Résolution avec Ipopt

In [ ]:
using JuMP, Ipopt

In [ ]:
x_low = [1.31,0.4314,0.6336,0.5,0.5,0.6,1.0,4.0,2.0,4.5689,5.0]
x_upp = [1.4,0.8,1.0,1.0,1.0,3.0,5.0,7.0,2.5,5.0,6.0]


x_saved =  [1.3344098963722457
 0.5572842161127423
 0.6757364753061974
 0.8291980513226953
 0.9233565833014519
 0.9588470511477797
 1.9610314699563896
 4.055321823656234
 2.048625993866472
 4.60296578920499
 5.95212572157736]

model = Model(Ipopt.Optimizer)
@variable(model,x_low[i] <= x[i=1:n] <= x_upp[i], start = x_saved[i])



@NLobjective(model, Min, sum((y[i] - (x[1]*exp(-x[5]*t[i]) + x[2]*exp(-x[6]*(t[i]-x[9])^2) + x[3]*exp(-x[7]*(t[i]-x[10])^2) + 
            x[4]*exp(-x[8]*(t[i]-x[11])^2)))^2 for i=1:m))

JuMP.optimize!(model)